In [1]:
import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["studentlife"]

In [3]:
student_uid = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']

In [4]:
# number of sms

In [5]:
df_sms = mydb.sms
df_sms = DataFrame(list(df_sms.find()))
df_sms = df_sms.drop("_id", axis=1)

df_sms['timestamp'] = pd.to_datetime(df_sms['timestamp'], unit='s')
df_sms['date'] = df_sms['timestamp'].dt.strftime('%Y-%m-%d')
df_sms['timestamp'] = df_sms['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(df_sms.head(3))

                                       ﻿id  \
0  c715f4c2-1e91-47eb-8aa4-977bb5e8b715-31   
1  649eb75d-33e7-4c56-88d0-2eec88e95e71-35   
2  649eb75d-33e7-4c56-88d0-2eec88e95e71-35   

                                 device            timestamp  \
0  027c6245-b07f-4492-94e8-036454303f9c  2013-03-27 02:21:29   
1  027c6245-b07f-4492-94e8-036454303f9c  2013-03-22 15:47:51   
2  027c6245-b07f-4492-94e8-036454303f9c  2013-03-22 15:47:51   

                                    MESSAGES_address  \
0                                                      
1  {"ONE_WAY_HASH":"953084ee813526fb5a363b8379450...   
2  {"ONE_WAY_HASH":"953084ee813526fb5a363b8379450...   

                                       MESSAGES_body  MESSAGES_date  \
0                                                                     
1  {"ONE_WAY_HASH":"77552e4c9cb80635bf3c8968f6d08...  1363967271656   
2  {"ONE_WAY_HASH":"f839bab8a85546d6e17881dcbf26d...  1363967249753   

  MESSAGES_locked                               

In [6]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

df_sms['timestamp'] = pd.to_datetime(df_sms['timestamp'])
df_sms['time'] = df_sms['timestamp'].dt.time

def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

df_sms['time_interval'] = df_sms['timestamp'].apply(categorize_time_interval)
df_sms = df_sms.drop("time", axis=1)
df_sms.head(3)

,﻿id,device,timestamp,MESSAGES_address,MESSAGES_body,MESSAGES_date,MESSAGES_locked,MESSAGES_person,MESSAGES_protocol,MESSAGES_read,MESSAGES_reply_path_present,MESSAGES_service_center,MESSAGES_status,MESSAGES_subject,MESSAGES_thread_id,MESSAGES_type,uid,date,time_interval
0,c715f4c2-1e91-47eb-8aa4-977bb5e8b715-31,027c6245-b07f-4492-94e8-036454303f9c,2013-03-27 02:21:29,,,,,,,,,,,,,,u58,2013-03-27,night
1,649eb75d-33e7-4c56-88d0-2eec88e95e71-35,027c6245-b07f-4492-94e8-036454303f9c,2013-03-22 15:47:51,"{""ONE_WAY_HASH"":""953084ee813526fb5a363b8379450...","{""ONE_WAY_HASH"":""77552e4c9cb80635bf3c8968f6d08...",1363967271656,False,,0,True,False,,0,,10,2,u58,2013-03-22,afternoon
2,649eb75d-33e7-4c56-88d0-2eec88e95e71-35,027c6245-b07f-4492-94e8-036454303f9c,2013-03-22 15:47:51,"{""ONE_WAY_HASH"":""953084ee813526fb5a363b8379450...","{""ONE_WAY_HASH"":""f839bab8a85546d6e17881dcbf26d...",1363967249753,False,"{""ONE_WAY_HASH"":""7c8d848f32fa1c53815556ee08aa8...",0,True,False,+12063130024,-1,,10,1,u58,2013-03-22,afternoon


In [7]:
print(len(df_sms))
df_sms = df_sms.dropna()
print(len(df_sms))
df_sms.columns

92584
36880


Index(['﻿id', 'device', 'timestamp', 'MESSAGES_address', 'MESSAGES_body',
       'MESSAGES_date', 'MESSAGES_locked', 'MESSAGES_person',
       'MESSAGES_protocol', 'MESSAGES_read', 'MESSAGES_reply_path_present',
       'MESSAGES_service_center', 'MESSAGES_status', 'MESSAGES_subject',
       'MESSAGES_thread_id', 'MESSAGES_type', 'uid', 'date', 'time_interval'],
      dtype='object')

In [8]:
len(df_sms['MESSAGES_address'].unique())
df_data_sms = DataFrame()

In [9]:
def calculate_sms_information(uid, df_sms):
    user_data = df_sms[df_sms['uid'] == uid]
    
    sms_info = []
    
    grouped_by_date = user_data.groupby('date')
    
    for date, group in grouped_by_date:
        unique_sms_morning = 0
        unique_sms_afternoon = 0
        unique_sms_evening = 0
        unique_sms_night = 0
        
        grouped_by_time_interval = group.groupby('time_interval')
        
        for time_interval, time_interval_group in grouped_by_time_interval:
            unique_sms_interval = time_interval_group['MESSAGES_address'].nunique()
            
            if time_interval == 'morning':
                unique_sms_morning = unique_sms_interval
            elif time_interval == 'afternoon':
                unique_sms_afternoon = unique_sms_interval
            elif time_interval == 'evening':
                unique_sms_evening = unique_sms_interval
            elif time_interval == 'night':
                unique_sms_night = unique_sms_interval
        sms_info.append([uid, date, unique_sms_morning, unique_sms_afternoon, unique_sms_evening, unique_sms_night])
    
    columns = ['uid', 'date', 'number of sms in morning', 'number of sms in afternoon', 'number of sms in evening', 'number of sms in night']
    result_df = pd.DataFrame(sms_info, columns=columns)
    
    return result_df

In [10]:
for uid in tqdm(student_uid):
    print(uid)
    df = calculate_sms_information(uid, df_sms)
    frames = [df_data_sms, df]
    df_data_sms = pd.concat(frames, ignore_index=True)

 22%|█████████▋                                 | 11/49 [00:00<00:00, 97.04it/s]

u00
u01
u02
u03
u04
u05
u07
u08
u09
u10
u12
u13
u14
u15
u16
u17
u18
u19
u20
u22
u23
u24
u25
u27
u30
u31
u32
u33
u34
u35
u36


100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 130.12it/s]

u39
u41
u42
u43
u44
u45
u46
u47
u49
u50
u51
u52
u53
u54
u56
u57
u58
u59


In [11]:
len(df_data_sms)

974

In [12]:
# number of calls

In [13]:
df_call = mydb.call_log
df_call = DataFrame(list(df_call.find()))
df_call = df_call.drop("_id", axis=1)

In [14]:
df_call['timestamp'] = pd.to_datetime(df_call['timestamp'], unit='s')
df_call['date'] = df_call['timestamp'].dt.strftime('%Y-%m-%d')
df_call['timestamp'] = df_call['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [15]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

df_call['timestamp'] = pd.to_datetime(df_call['timestamp'])
df_call['time'] = df_call['timestamp'].dt.time

def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

df_call['time_interval'] = df_call['timestamp'].apply(categorize_time_interval)
df_call = df_call.drop("time", axis=1)
df_call.head(3)

,﻿id,device,timestamp,CALLS__id,CALLS_date,CALLS_duration,CALLS_name,CALLS_number,CALLS_numberlabel,CALLS_numbertype,CALLS_type,uid,date,time_interval
0,4ab7e35d-026c-487f-ab00-5acf91bce8c3-40,661f40c2-3f4a-412b-9d04-cf49dd4bfe93,2013-03-27 01:48:55,,,,,,,,,u57,2013-03-27,night
1,2c8deb53-7d0a-453d-94cc-b4af9ee5fd05-22,661f40c2-3f4a-412b-9d04-cf49dd4bfe93,2013-03-24 19:38:05,23908,1364153885796,33,"{""ONE_WAY_HASH"":""efac29c054f9ce222c9fa485441fe...","{""ONE_WAY_HASH"":""ebcea50db44772a5a7287160c2589...",,"{""ONE_WAY_HASH"":""da4b9237bacccdf19c0760cab7aec...",1,u57,2013-03-24,evening
2,ebf1cb59-e1bb-445d-ae70-b19758165f77-138,661f40c2-3f4a-412b-9d04-cf49dd4bfe93,2013-03-27 02:48:55,,,,,,,,,u57,2013-03-27,night


In [16]:
print(len(df_call))
df_call.isnull().sum()

67173


﻿id                      0
device                   0
timestamp                0
CALLS__id            37550
CALLS_date           37550
CALLS_duration       37550
CALLS_name           37550
CALLS_number         37550
CALLS_numberlabel    37550
CALLS_numbertype     37550
CALLS_type           37550
uid                      0
date                     0
time_interval            0
dtype: int64

In [17]:
df_call = df_call.dropna()
print(len(df_call))
df_call.columns

29623


Index(['﻿id', 'device', 'timestamp', 'CALLS__id', 'CALLS_date',
       'CALLS_duration', 'CALLS_name', 'CALLS_number', 'CALLS_numberlabel',
       'CALLS_numbertype', 'CALLS_type', 'uid', 'date', 'time_interval'],
      dtype='object')

In [18]:
len(df_call['CALLS__id'].unique())
df_data_call = DataFrame()

In [19]:
def calculate_call_information(uid, df_call):
    user_data = df_call[df_call['uid'] == uid]
    
    call_info = []
    
    grouped_by_date = user_data.groupby('date')
    
    for date, group in grouped_by_date:
        unique_call_morning = 0
        unique_call_afternoon = 0
        unique_call_evening = 0
        unique_call_night = 0
        
        grouped_by_time_interval = group.groupby('time_interval')
        
        for time_interval, time_interval_group in grouped_by_time_interval:
            unique_call_interval = time_interval_group['CALLS__id'].nunique()
            
            if time_interval == 'morning':
                unique_call_morning = unique_call_interval
            elif time_interval == 'afternoon':
                unique_call_afternoon = unique_call_interval
            elif time_interval == 'evening':
                unique_call_evening = unique_call_interval
            elif time_interval == 'night':
                unique_call_night = unique_call_interval
        
        call_info.append([uid, date, unique_call_morning, unique_call_afternoon, unique_call_evening, unique_call_night])
    
    columns = ['uid', 'date', 'number of call in morning', 'number of call in afternoon', 'number of call in evening', 'number of call in night']
    result_df = pd.DataFrame(call_info, columns=columns)
    
    return result_df

In [20]:
for uid in tqdm(student_uid):
    print(uid)
    df = calculate_call_information(uid, df_call)
    frames = [df_data_call, df]
    df_data_call = pd.concat(frames, ignore_index=True)

 24%|██████████▎                               | 12/49 [00:00<00:00, 102.15it/s]

u00
u01
u02
u03
u04
u05
u07
u08
u09
u10
u12
u13
u14
u15
u16
u17
u18
u19
u20
u22
u23
u24
u25
u27
u30
u31
u32
u33
u34
u35
u36
u39
u41
u42
u43
u44
u45
u46


 78%|████████████████████████████████▌         | 38/49 [00:00<00:00, 183.17it/s]

u47
u49
u50
u51
u52
u53
u54
u56
u57
u58
u59


100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 111.89it/s]


In [21]:
print(len(df_data_call))
df_data_call.head(3)

1331


,uid,date,number of call in morning,number of call in afternoon,number of call in evening,number of call in night
0,u00,2013-03-23,0,0,1,0
1,u00,2013-03-24,1,1,2,1
2,u00,2013-03-25,1,2,2,1


In [22]:
print(len(df_data_call))
df_data_call = df_data_call.dropna()
print(len(df_data_call))

1331
1331


In [23]:
# number of Bluetooth contacts

In [24]:
df_blue = mydb.bluetooth
df_blue = DataFrame(list(df_blue.find()))
df_blue = df_blue.drop("_id", axis=1)

df_blue['timestamp'] = pd.to_datetime(df_blue['timestamp'], unit='s')
df_blue['date'] = df_blue['timestamp'].dt.strftime('%Y-%m-%d')
df_blue['timestamp'] = df_blue['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [25]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

df_blue['timestamp'] = pd.to_datetime(df_blue['timestamp'])
df_blue['time'] = df_blue['timestamp'].dt.time

def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

df_blue['time_interval'] = df_blue['timestamp'].apply(categorize_time_interval)
df_blue = df_blue.drop("time", axis=1)
df_blue.head(3)

,MAC,class_id,level,uid,timestamp,date,time_interval
0,00:1E:C2:8F:C8:E1,3670276,-84,u45,2013-03-27 18:57:49,2013-03-27,evening
1,00:1E:C2:8F:C8:E1,3670276,-72,u45,2013-03-27 19:18:10,2013-03-27,evening
2,00:1E:C2:8F:C8:E1,3670276,-77,u45,2013-04-01 21:45:55,2013-04-01,evening


In [26]:
print(len(df_blue))
df_blue.isnull().sum()

250695


MAC              0
class_id         0
level            0
uid              0
timestamp        0
date             0
time_interval    0
dtype: int64

In [27]:
def calculate_bluetooth_information(uid, df_blue):
    user_data = df_blue[df_blue['uid'] == uid]
    
    blue_info = []
    
    grouped_by_date = user_data.groupby('date')
    
    for date, group in grouped_by_date:
        unique_bluetooth_morning = 0
        unique_bluetooth_afternoon = 0
        unique_bluetooth_evening = 0
        unique_bluetooth_night = 0
        
        grouped_by_time_interval = group.groupby('time_interval')
        
        for time_interval, time_interval_group in grouped_by_time_interval:
            unique_bluetooth_contacts_interval = time_interval_group['MAC'].nunique()
            
            if time_interval == 'morning':
                unique_bluetooth_morning = unique_bluetooth_contacts_interval
            if time_interval == 'afternoon':
                unique_bluetooth_afternoon = unique_bluetooth_contacts_interval
            elif time_interval == 'evening':
                unique_bluetooth_evening = unique_bluetooth_contacts_interval
            elif time_interval == 'night':
                unique_bluetooth_night = unique_bluetooth_contacts_interval
        
        blue_info.append([uid, date, unique_bluetooth_morning, unique_bluetooth_afternoon, unique_bluetooth_evening, unique_bluetooth_night])
    
    columns = ['uid', 'date', 'number of Bluetooth contacts morning', 'number of Bluetooth contacts afternoon', 'number of Bluetooth contacts evening', 'number of Bluetooth contacts night']
    result_df = pd.DataFrame(blue_info, columns=columns)
    
    return result_df

In [28]:
print(len(df_blue))
df_data_bluetooth = DataFrame()

250695


In [29]:
for uid in tqdm(student_uid):
    print(uid)
    df = calculate_bluetooth_information(uid, df_blue)
    frames = [df_data_bluetooth, df]
    df_data_bluetooth = pd.concat(frames, ignore_index=True)

  6%|██▋                                         | 3/49 [00:00<00:01, 27.05it/s]

u00
u01
u02
u03
u04
u05


 12%|█████▍                                      | 6/49 [00:00<00:01, 27.66it/s]

u07


 18%|████████                                    | 9/49 [00:00<00:01, 27.83it/s]

...

 92%|███████████████████████████████████████▍   | 45/49 [00:01<00:00, 28.82it/s]

u52
u53
u54
u56
u57


 98%|██████████████████████████████████████████ | 48/49 [00:01<00:00, 28.40it/s]

u58
u59


100%|███████████████████████████████████████████| 49/49 [00:01<00:00, 28.57it/s]


In [30]:
print(len(df_data_bluetooth))
df_data_bluetooth.head(3)

2728


,uid,date,number of Bluetooth contacts morning,number of Bluetooth contacts afternoon,number of Bluetooth contacts evening,number of Bluetooth contacts night
0,u00,2013-03-27,0,8,6,0
1,u00,2013-03-28,0,3,7,0
2,u00,2013-03-29,0,10,7,2


In [31]:
# total time in phone conversation

In [32]:
df_con = mydb.conversation
df_con = DataFrame(list(df_con.find()))
df_con = df_con.drop("_id", axis=1)

df_con['conversation duration'] = (df_con['end_timestamp'] - df_con['timestamp']) / 60

df_con['timestamp'] = pd.to_datetime(df_con['timestamp'], unit='s')
df_con['date'] = df_con['timestamp'].dt.strftime('%Y-%m-%d')
df_con['timestamp'] = df_con['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [33]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

df_con['timestamp'] = pd.to_datetime(df_con['timestamp'])
df_con['time'] = df_con['timestamp'].dt.time

def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

df_con['time_interval'] = df_con['timestamp'].apply(categorize_time_interval)
df_con = df_con.drop("time", axis=1)
df_con.head(3)

,uid,timestamp,end_timestamp,conversation duration,date,time_interval
0,u10,2013-03-27 04:42:41,1364359583,3.700000,2013-03-27,night
1,u10,2013-03-27 04:50:14,1364359894,1.333333,2013-03-27,night
2,u10,2013-03-27 13:32:32,1364391404,4.200000,2013-03-27,afternoon


In [34]:
df_con.isnull().sum()

uid                      0
timestamp                0
end_timestamp            0
conversation duration    0
date                     0
time_interval            0
dtype: int64

In [35]:
def calculate_conversation_information(uid, df_con):
    user_data = df_con[df_con['uid'] == uid]
    
    con_info = []
    
    grouped_by_date = user_data.groupby('date')
    
    for date, group in grouped_by_date:
        mean_con_morning = 0
        mean_con_afternoon = 0
        mean_con_evening = 0
        mean_con_night = 0
        
        grouped_by_time_interval = group.groupby('time_interval')
        
        for time_interval, time_interval_group in grouped_by_time_interval:
            mean_con_interval = time_interval_group['conversation duration'].mean()
            
            if time_interval == 'morning':
                mean_con_morning = mean_con_interval
            elif time_interval == 'afternoon':
                mean_con_afternoon = mean_con_interval
            elif time_interval == 'evening':
                mean_con_evening = mean_con_interval
            elif time_interval == 'night':
                mean_con_night = mean_con_interval
        
        con_info.append([uid, date, mean_con_morning, mean_con_afternoon, mean_con_evening, mean_con_night])
    
    columns = ['uid', 'date', 'conversation in morning', 'conversation in afternoon', 'conversation in evening', 'conversation in night']
    result_df = pd.DataFrame(con_info, columns=columns)
    
    return result_df

In [36]:
print(len(df_con))
df_data_conversation = DataFrame()

79023


In [37]:
for uid in tqdm(student_uid):
    print(uid)
    df = calculate_conversation_information(uid, df_con)
    frames = [df_data_conversation, df]
    df_data_conversation = pd.concat(frames, ignore_index=True)

 10%|████▍                                       | 5/49 [00:00<00:00, 47.02it/s]

u00
u01
u02
u03
u04
u05
u07
u08
u09
u10
u12


 35%|██████████████▉                            | 17/49 [00:00<00:00, 51.98it/s]

u13
u14
u15
u16
u17
u18
u19
u20
u22
u23
u24
u25
u27


 61%|██████████████████████████▎                | 30/49 [00:00<00:00, 56.23it/s]

u30
u31
u32
u33
u34
u35
u36
u39
u41
u42
u43
u44
u45


 88%|█████████████████████████████████████▋     | 43/49 [00:00<00:00, 58.16it/s]

u46
u47
u49
u50
u51
u52
u53
u54
u56
u57
u58
u59


100%|███████████████████████████████████████████| 49/49 [00:00<00:00, 55.80it/s]


In [38]:
# app_usage

In [39]:
df_data_app = pd.read_csv('recreaing_social_appusage_features.csv')
print(len(df_data_app))

3083


In [40]:
print(len(df_data_app))
print(len(df_data_sms))
print(len(df_data_call))
print(len(df_data_bluetooth))
print(len(df_data_conversation))

3083
974
1331
2728
2732


In [41]:
# merged_df = pd.merge(df_data_app, df_data_sms, on=['uid', 'date'], how='inner')

# merged_df = pd.merge(merged_df, df_data_call, on=['uid', 'date'], how='inner')

merged_df = pd.merge(df_data_app, df_data_call, on=['uid', 'date'], how='inner')

merged_df = pd.merge(merged_df, df_data_bluetooth, on=['uid', 'date'], how='inner')

df = pd.merge(merged_df, df_data_conversation, on=['uid', 'date'], how='inner')

In [42]:
print(df['uid'].unique())
len(df)

['u00' 'u02' 'u08' 'u12' 'u13' 'u24' 'u31' 'u36' 'u46' 'u47' 'u49' 'u50'
 'u51' 'u52' 'u53' 'u54' 'u56' 'u57' 'u58' 'u59']


1119

In [43]:
df.to_csv('recreating_social_features_studentlife.csv')